In [ ]:
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from scipy.stats import spearmanr, pearsonr
import geopandas as gp
from shapely import wkt
from matplotlib.colors import ListedColormap
import math
from colorsys import hls_to_rgb
import folium

# import geoplot

In [ ]:
from IPython.display import Markdown as md

In [ ]:
mp.rcParams.update({'font.size': 19})
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Read Data 

In [ ]:
data_dir = "data/" 

In [ ]:
fp_forecast_regions = data_dir+"nws_forecast_regions.csv"
fp_tornado_paths = data_dir+"tornado_paths.csv"
fp_storms_2021 = data_dir+"storms_2021.csv"

In [ ]:
forecast_regions = gp.read_file(fp_forecast_regions, index_col=0, ignore_geometry=True)
tornado_paths = pd.read_csv(fp_tornado_paths, index_col=0)

In [ ]:
fp_counties = data_dir+"counties/c_22mr22.dbf"
counties = gp.read_file(fp_counties, ignore_geometry=False)

In [ ]:
storms_2021 = pd.read_csv(fp_storms_2021, index_col=0)

In [ ]:
tornado_paths.head()

In [ ]:
pd.set_option('display.max_columns', None)
storms_2021.head()

In [ ]:
forecast_regions.head()

In [ ]:
counties.head()

In [ ]:
storms_2021.shape

In [ ]:
states = {"AL":"Alabama","AK":"Alaska", "AS":"American samoa","AZ":"Arizona","AR":"Arkansas",
          "CA":"California","CO":"Colorado","CT":"Connecticut",
          "DE":"Delaware", "DC": "District of Columbia",
          "FL":"Florida", "FM": "Micronesia",
          "GA":"Georgia", "GU":"Guam",
          "HI":"Hawaii",
          "ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa",
          "KS":"Kansas","KY":"Kentucky",
          "LA":"Louisiana",
          "ME":"Maine","MD":"Maryland","MA":"Massachusetts",
          "MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri",
          "MT":"Montana", "MH":"Marshall Islands", "MP": "Northern Marina Islands",

          "NE":"Nebraska","NV":"Nevada","NH":"New hampshire","NJ":"New jersey","NM":"New mexico",
          "NY":"New york","NC":"North carolina","ND":"North dakota",
          "OH":"Ohio","OK":"Oklahoma","OR":"Oregon",
          "PA":"Pennsylvania", "PR":"Puerto rico", "PW":"Palau",
          "RI":"Rhode island",
          "SC":"South carolina","SD":"South dakota",
          "TN":"Tennessee","TX":"Texas",
          "UT":"Utah",
          "VT":"Vermont","VA":"Virginia","VI":"Virgin islands",
          "WA":"Washington","WV":"West virginia","WI":"Wisconsin","WY":"Wyoming"}

In [ ]:
storms_states = storms_2021["state"].unique()

In [ ]:
forecast_states = [states[x].lower() for x in forecast_regions["state"].unique()]
len(np.sort(forecast_states))

In [ ]:
excluded = []
for s in storms_2021["state"].unique():
    if s.strip().lower() not in forecast_states:
        print(s.lower())
        excluded.append(s)

# Preprocessing Data

In [ ]:
counties.rename(columns={
    "STATE": "state",
    "CWA": "cwa",
    "TIME_ZONE": "time_zone",
    "FIPS": "zone",
    "COUNTYNAME": "name",
    "FE_AREA": "fe_area",
    "LON": "lon",
    "LAT": "lat",
    "geometry": "shape_geometry"
}, inplace=True)

In [ ]:
counties["zone"] = counties["zone"].apply(lambda x:x[-3:])

In [ ]:
counties["type"] = "C"

In [ ]:
counties.head()

In [ ]:
counties.shape

## Forecast Regions

In [ ]:
forecast_regions["type"] = "Z"

In [ ]:
forecast_regions.shape

In [ ]:
forecast_regions['shape_geometry'] = forecast_regions['shape_geometry'].apply(wkt.loads)

In [ ]:
forecast_regions = pd.concat([forecast_regions, counties], join="inner")

In [ ]:
forecast_regions["zone"] = forecast_regions["zone"].astype(int)

In [ ]:
forecast_regions["state_name"] = forecast_regions["state"].apply(lambda x:states[x].lower())

In [ ]:
forecast_regions["name"] = forecast_regions["name"].apply(lambda x:x.lower())

In [ ]:
forecast_regions.head()

In [ ]:
forecast_regions.shape

In [ ]:
forecast_regions.drop_duplicates(subset = ["state","cwa","zone","type"], keep="first",
                                inplace=True)

In [ ]:
forecast_regions = forecast_regions.set_geometry("shape_geometry")
forecast_regions = forecast_regions.set_crs("EPSG:4326")

In [ ]:
forecast_regions.crs

In [ ]:
forecast_regions[forecast_regions["type"] == "C"].explore(column="state",legend=False)

In [ ]:
forecast_regions[forecast_regions["type"] == "Z"].explore(column="state", legend=False)

## Handling Multiple Timezones in the Storm Data

In [ ]:
def convert_std_tz(tz):
    tmp = re.sub(r'[A-Z]+','',tz)
    res = tmp if tmp.startswith("-") else "+"+tmp
    return res

In [ ]:
storms_2021["event_timezone"] = storms_2021["event_timezone"].apply(convert_std_tz)



In [ ]:
storms_2021["event_begin_time_utc"] = pd.to_datetime(
    storms_2021["event_begin_time"]+storms_2021["event_timezone"],
    utc=True)

In [ ]:
storms_2021["event_end_time_utc"] = pd.to_datetime(
    storms_2021["event_end_time"]+storms_2021["event_timezone"],
    utc=True)

## Other Preprocessing

In [ ]:
storms_2021["cz_name"] = storms_2021["cz_name"].apply(lambda x:x.lower())
storms_2021["state"] = storms_2021["state"].apply(lambda x:x.lower())

In [ ]:
storms_2021["event_duration"] = storms_2021["event_end_time_utc"] - storms_2021["event_begin_time_utc"]
storms_2021["event_duration"] = storms_2021["event_duration"]/pd.Timedelta("1 hour")

In [ ]:
storms_2021["injuries_total"] = storms_2021["injuries_direct"]+storms_2021["injuries_indirect"]
storms_2021["deaths_total"] = storms_2021["deaths_direct"]+storms_2021["deaths_indirect"]

# Damage Analysis

The most important thing about a storm is the damage caused by it. Lets see if there is a correlation between the event duration and the number of casualties reported.

In [ ]:
storms_2021[
    ["injuries_direct", "injuries_indirect",
     "deaths_direct", "deaths_indirect", 
     "damage_property", "damage_crops"]
].describe()

As can be seen from the summary statistics, the storms report no damage (human or material) up to the 75th percentile. Beyond the 75th percentile, they resulted in more material damage than human life.

In [ ]:
storms_melt = pd.melt(storms_2021, id_vars = ["event_type", "episode_id", "state", "event_duration"], 
                  value_vars = ["injuries_total","deaths_total"],
                  ignore_index=False,
                  var_name='casualty_type', value_name='value')

In [ ]:
storms_melt = storms_melt[storms_melt["value"] > 0]
storms_melt.describe()

In [ ]:
ax = sns.scatterplot(data=storms_melt,
                     x="event_duration",y="value",hue="casualty_type")
ax.set_title("Event Duration Vs. Number of Casualties")
ax.set_ylim((1,100))
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

Most storm events of 2021 reported no deaths. Let us take a closer look at the injuries reported.

In [ ]:
ax = sns.scatterplot(data=storms_2021,x="event_duration",y="injuries_total")
ax.set_title("Event Duration Vs. Number of Injuries")
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

Interestingly, the longest storms caused the least injuries. Let us look at the numerical correlation between these two variables.

In [ ]:
spearman_corr_coef = spearmanr(storms_2021["event_duration"], storms_2021["injuries_total"])
spearman_corr_coef

In [ ]:
spearman_corr_coef = spearmanr(storms_2021["event_duration"], storms_2021["deaths_total"])
spearman_corr_coef

Lets now look at the damage caused to crops and property.

In [ ]:
ax = sns.scatterplot(data=storms_2021,x="event_duration",y="damage_property")
ax.set_title("Event Duration Vs. Property Damage")
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

# Analysis of Different Storm Types

In [ ]:
num_storm_types = len(storms_2021["event_type"].unique())
md("{} different types of storms were reported in 2021".format(num_storm_types))

In [ ]:
gb_atts = ["event_type","state", "event_begin_time_utc","event_end_time_utc"]
atts = ["episode_id","event_type","state","cz_name", "event_begin_time_utc", "event_end_time_utc",
        "injuries_direct", "injuries_indirect", "injuries_total", 
        "deaths_direct","deaths_indirect","deaths_total",
        "damage_property",
       "damage_crops","event_duration"]

In [ ]:
gb = storms_2021[atts].groupby(gb_atts).agg({
    "episode_id":lambda x:list(x.unique()),
    "cz_name":lambda x:list(x.unique()),
    "injuries_direct": lambda x:x.dropna().sum(),
    "injuries_indirect": lambda x:x.dropna().sum(),
    "injuries_total": lambda x:x.sum(),
    "deaths_direct": lambda x:x.dropna().sum(),
    "deaths_indirect": lambda x:x.dropna().sum(),
    "deaths_total": lambda x:x.sum(),
    "damage_property":lambda x:x.dropna().sum(),
    "damage_crops": lambda x:x.dropna().sum(),
#     "event_end_time_utc": lambda x:np.max(x),
    "event_duration": lambda x: np.max(x)
})

In [ ]:
gb.reset_index(inplace=True)

In [ ]:
gb.head()

In [ ]:
storms_2021[(storms_2021["event_type"]=="excessive heat") & (storms_2021["state"] == "Idaho")]

In [ ]:
gb[(gb["event_type"]=="excessive heat") & (gb["state"] == "Idaho")]

In [ ]:
ax = sns.barplot(data=gb,x="event_type",y="event_duration")
plt.xticks(rotation=90)
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

In [ ]:
top_events = list(gb.groupby("event_type").agg({
    "event_duration":np.mean
}).reset_index().sort_values(by="event_duration").iloc[-5:]["event_type"])

In [ ]:
top_events

Drought and Wildfire had an average duration greater than 200 hours, while all other storms lasted for less than a 100 hours (~4 days). We now analyze their duration by state.

In [ ]:
dat = gb[gb["event_type"].isin(top_events[-2:])]
ax = sns.boxplot(data=dat, y="state",x="event_duration",hue="event_type", orient="h")
plt.xticks(rotation=90)
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 20)
plt.show()

Wildfire is an event type that can be controlled through human intervention. Drought, on the other hand, is a noatural calamity and very little can be done to control it. Therefore, it makes sense that in states where both occurred, the droughts lasted longer than the wildfire.

In [ ]:
ax = sns.boxplot(data=gb[
    (gb["event_type"].isin(top_events[:-2])) & 
    (gb["event_duration"]!=0)],
                 y="state",
                 x="event_duration",
                 hue="event_type",
                 orient="h")
plt.xticks(rotation=90)
plt.grid(color='grey', linestyle='--')
ax.set_title("Storm Duration by State")
fig = plt.gcf()
fig.set_size_inches(15, 20)
plt.show()

In [ ]:
gb_melt = pd.melt(gb[gb["event_type"].isin(top_events)],
                  id_vars = ["event_type", "episode_id", "state", "event_duration"], 
                  value_vars = ["injuries_total","deaths_total"],
                  ignore_index=False,
                  var_name='casualty_type', value_name='value')



In [ ]:
gb_melt.loc[0]

In [ ]:
gb_melt[gb_melt["value"]==np.max(gb_melt["value"])]

In [ ]:
gb_melt[(gb_melt["event_type"]=="drought") & (gb_melt["value"]>0)]

Less than one casualty was reported for most events. The maximum casualties were reported as 109 deaths, in Oregon due to excessive heat over 78 hours.


In [ ]:
ax = sns.barplot(data=gb_melt,
                 x="event_type",y="value",hue="casualty_type",
                 estimator=np.max, 
                 ci=None)
plt.xticks(rotation=90)
ax.set_title("Maximum Casualties per Storm Type")
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

While drought had the longest duration, most damage was caused by excessive heat.

# Which Zones took the Longest Hit?

In [ ]:
storms_2021[(storms_2021["cz_type"]=="Z")].shape

In [ ]:
forecast_regions.head()

In [ ]:
storms_combos = storms_2021[
    ["state","cz_fips_code","episode_id"]
].groupby(["state","cz_fips_code"],as_index=False).count()[
    ["state","cz_fips_code"]].to_records(index=False)

In [ ]:
storms_combos = set((x,y) for (x,y) in storms_combos)

In [ ]:
forecast_regions.head()

In [ ]:
regions_combos = set( (x,y) for (x,y) in forecast_regions[["state_name","zone"]].to_records(index=False))

In [ ]:
len(storms_combos.intersection(regions_combos))

In [ ]:
tmp = forecast_regions[["state_name","type","zone","shape_geometry"]].groupby(["state_name","type","zone"]).count()
tmp[tmp["shape_geometry"] > 1]

In [ ]:
forecast_regions[(forecast_regions["state_name"] == "micronesia") &
                (forecast_regions["type"] == "C") &
                (forecast_regions["zone"] == 1)]

In [ ]:
merged = storms_2021[storms_2021["cz_type"]=="Z"][[
    "episode_id", "event_id", "state", "state_fips_code",
    "event_type", "cz_fips_code", "cz_name", "cz_type",
    "event_begin_time_utc", "event_end_time_utc",
    "injuries_direct", "injuries_indirect", "injuries_total",
    "deaths_direct", "deaths_indirect", "deaths_total",
    "damage_property", "damage_crops",
    "event_duration"
]].merge(forecast_regions[forecast_regions["type"]=="Z"][[
    "state","state_name","zone","type","name","shape_geometry"
]],left_on=["state", "cz_type","cz_fips_code"],
         right_on=["state_name","type","zone"],
        validate="m:1")
merged.head()

In [ ]:
merged.shape

In [ ]:
storms_2021[
    (storms_2021["state"]=="california") &
    (storms_2021["cz_type"]=="Z") &
    (storms_2021["cz_fips_code"] == 304)
]

In [ ]:
forecast_regions[(forecast_regions["state_name"] == "california") & 
                 (forecast_regions["type"] == "Z") &
                 (forecast_regions["zone"]==304)]

In [ ]:
merged[(merged["name"].isna()) & (merged["state_x"]=="california")]

In [ ]:
merged_gb = merged[
    ["episode_id" ,"state_name", "event_type",
    "zone", "name", "event_duration",
     "shape_geometry",
    "injuries_total",
    "deaths_total",
    "damage_property",
    "damage_crops"]
].groupby(["event_type", "state_name","zone"]).agg({
    "episode_id":lambda x:len(x.unique()),
    "name": lambda x:x.iloc[0],
    "event_duration": lambda x:np.mean(x),
    "shape_geometry": lambda x:x.iloc[0],
    "injuries_total": lambda x:x.sum(),
    "deaths_total": lambda x:x.sum(),
    "damage_property": lambda x:x.dropna().sum(),
    "damage_crops": lambda x:x.dropna().sum()
    
})
merged_gb.shape

In [ ]:
merged_gb.rename(columns={"episode_id":"count_episodes",
                 "event_duration":"mean_event_duration"}, inplace=True)

In [ ]:
merged_gb.head()

In [ ]:

merged_gb_geo = gp.GeoDataFrame(merged_gb, geometry="shape_geometry", crs="EPSG:4326")
merged_gb_geo.crs

In [ ]:
merged_gb_geo.loc[(slice(None),"south carolina",slice(None)),:]

In [ ]:
event_type_name = "excessive heat"
color_by = "injuries_total" #"mean_event_duration"

In [ ]:
# [37.0902,-95.7129]
m=folium.Map(max_bounds=True)
sw = forecast_regions[['lat', 'lon']].min().values.tolist()
ne = forecast_regions[['lat', 'lon']].max().values.tolist()
# m.fit_bounds([sw,ne])
for _, r in forecast_regions.iterrows():
    sim_geo = gp.GeoSeries(r['shape_geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'color':'black', 'weight':0.5})
#     folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(m)
m = merged_gb_geo.loc[(event_type_name,slice(None)),:].explore(column=color_by, m=m)

In [ ]:
m

Interesting Observations:
- The duration of drought in New Mexico appears significantly higher than that of any other state. Lets compare New Mexico to a few other randomly chosen states.
- A maximum of 88 deaths were recorded in the "greater portland metro area", over 2 episodes of excessive heat. The neighbouring zone of "central willamette valley" also saw 19 deaths.
- 45 injuries due to excessive heat were reported in Tulsa over 5 episodes, while a single episode caused 9 injuries in St. Louis

## Drought In 2019

In [ ]:
dt = storms_2021[storms_2021["event_type"]=="drought"]
dt.head()

In [ ]:
ax = sns.displot(dt,x="event_duration",hue="episode_id", legend=None)
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

In [ ]:
ax = sns.displot(dt,x="event_duration",hue="state")
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

### New Mexico vs Random Subset

In [ ]:
states = list(dt[dt["state"]!="New mexico"]["state"].unique())

In [ ]:
import random
ax = sns.displot(dt[dt["state"].isin(["New mexico"]+random.sample(states,5))],x="event_duration",hue="state")
plt.grid(color='grey', linestyle='--')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

### Analysing duration of different episodes of drought in New Mexico

In [ ]:
dt_nm = dt[dt["state"]=="New mexico"]

In [ ]:
print(dt_nm.shape)
dt_nm.head()

In [ ]:
len(dt_nm["episode_id"].unique())

There were 22 episodes of drought reported in New Mexico in 2021

In [ ]:
dt_nm

# Per Episode Analysis

- Which zones suffered the most storm episodes in 2021 (in general)?
- What is the average number of zones spanned by an episode?
- Casualties per episode

In [ ]:
storms_2021[(storms_2021["cz_fips_code"] == 1) & (storms_2021["state"] == "south carolina")]

In [ ]:
forecast_regions[(forecast_regions["state_name"] == "south carolina") & (forecast_regions["zone"] == 1)]

In [ ]:
forecast_regions[(forecast_regions["name"] == "abbeville")] 

In [ ]:
zones_gb = storms_2021[["state","cz_fips_code", "cz_name", "episode_id"]].groupby("cz_name", as_index=False).agg(
{
    "state": lambda x:x.iloc[0],
    "cz_fips_code": lambda x:x.iloc[0],
    "episode_id": lambda x:len(x.unique())
})

In [ ]:
zones_gb

In [ ]:
zones_gb[zones_gb["cz_name"] == "abbeville"]

In [ ]:
excluded

In [ ]:
# Remove "zones" for which we do not have geometry.

zones_with_geom = zones_gb.merge(forecast_regions[[
    "state","state_name","zone","name","shape_geometry"
]],left_on=["state","cz_fips_code"],
         right_on=["state_name","zone"])

zones_with_geom.head()

In [ ]:
def transform_df(df):
#     print(df)
#     input()
    df["event_id"] = len(df["event_id"])
    df["injuries_total"] = np.sum(df["injuries_total"])
    df["deaths_total"] = np.sum(df["deaths_total"])
    return df

In [ ]:
gb_event_2021 = storms_2021[
    ["episode_id","event_id","event_type","state","cz_name","cz_fips_code",
    "injuries_total", "deaths_total"]
].groupby("episode_id").apply(transform_df)

#  {
#         "event_type": lambda x:x.iloc[0],
#         "event_id": "count",
#         "state": lambda x:list(x.unique()),
#         "cz_name": lambda x:list(x.unique()),
#         "cz_fips_code": lambda x:list(x.unique()),
#         "injuries_total": "sum",
#         "deaths_total": "sum"
#     }

gb_event_2021.rename(columns={"event_id":"count_reported",
                             "cz_name":"zones",
                             "cz_fips_code":"zones_fips_code"},inplace=True)

In [ ]:
print(gb_event_2021.shape)
gb_event_2021.head(7)

8378 storm episodes were reported in 2021

In [ ]:
ax = sns.countplot(x="event_type", data=storms_2021)
plt.xticks(rotation=90)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

"Thunderstorm wind" had the largest number of episodes reported.